# 환경설정

In [3]:
import pandas as pd

In [4]:
print('Pandas Version :', pd.__version__)

Pandas Version : 1.3.4


# KOSPI200 리스트 로드

https://markets.hankyung.com/indices/kospi200

In [4]:
kospi200 = pd.read_csv("붙임2212KOSPI200KOSDAQ150및KRX300정기변경내역.csv")

In [5]:
kospi200

,종목명
0,메리츠화재
1,하이트진로
2,유한양행
3,CJ대한통운
4,두산
...,...
195,DL이앤씨
196,카카오페이
197,케이카
198,F&F


# 네이버뉴스크롤링

In [11]:
import requests
import bs4
from bs4 import BeautifulSoup

In [13]:
print('Requests Version :', requests.__version__)
print('BeautifulSoup :', bs4.__version__)

Requests Version : 2.26.0
BeautifulSoup : 4.10.0


In [11]:
keyword = []
dates = []
presses = []
titles = []
summs = []
links = []

In [12]:
start_index = 0
end_index = 199

In [ ]:
for i in range(start_index, end_index+1):
  print(str(i)+" "+kospi200["종목명"][i]+"\n")
  # 검색어 지정
  my_keyword = kospi200["종목명"][i]
  my_start_date1 = "2022.01.01"
  my_start_date2 = my_start_date1.replace('.','')
  my_end_date1 = "2022.12.24"
  my_end_date2 = my_end_date1.replace('.','')
  my_number = 4000

  # 페이지번호 지정
  num_list =[]
  for num in range(0,my_number,10):
      num_list.append(num)

  # 뉴스 영역 가져오기
  articles = []
  for number in num_list:
      URL = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search_word}&sort=0&photo=0&field=0&pd=3&ds={startdate}&de={enddate}&cluster_rank=37&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{start_date}to{end_date},a:all&start={page_num}'
      headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
          'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'}
      url = URL.format(search_word = my_keyword ,startdate= my_start_date1,enddate = my_end_date1,
                                    start_date = my_start_date2,end_date = my_end_date2 ,page_num = number)
      
      response = requests.get(url, headers = headers)
      soup=BeautifulSoup(response.text,'lxml')
      news_area = soup.find_all('div', {'class' : 'news_area'})
      articles += news_area

  # 1차원 배열에 차곡차곡 쌓아 넣기
  for article in articles:
        try:
            article_link = article.find('a', attrs = {'class' : 'news_tit'})['href']
            article_title = article.find('a', attrs = {'class' : 'news_tit'})['title']
            article_summ = article.find('a', attrs = {'class' : 'api_txt_lines dsc_txt_wrap'}).get_text()
            date2 = article.find('span', attrs = {'class' : 'info'}).get_text()
            press2 = article.find('a', {'class': 'info press'}).get_text()
            press2 = press2.replace("언론사 선정",'')
            keyword.append(my_keyword)
            dates.append(date2)
            presses.append(press2)
            titles.append(article_title)
            summs.append(article_summ)
            links.append(article_link)
        except:
            print("error")

In [14]:
import pandas as pd
df = pd.DataFrame({'검색어': keyword,
                   '날짜': dates,
                   '언론사': presses,
                   '제목': titles,
                   '요약': summs,
                   '링크' : links})

In [17]:
df

,검색어,날짜,언론사,제목,요약,링크
0,메리츠화재,3일 전,머니S,"김용범 메리츠화재 대표, '영업통' 김경환 부사장 앉혔다… 이유는?",김용범 메리츠화재 대표가 김경환 전무를 부사장으로 승진시키는 등 지난 23일 임원인...,http://moneys.mt.co.kr/news/mwView.php?no=2022...
1,메리츠화재,4일 전,매경이코노미,내년 車 보험료 내린다…현대·KB·메리츠화재 등 줄 인하,"롯데손보 2.9%로 인하폭 가장 커 메리츠 2.5%, 현대·KB 2.0% 등 현대해...",https://www.mk.co.kr/article/10578542
2,메리츠화재,3일 전,팍스넷뉴스,"메리츠證, 장원재 사장 승진…""리스크관리 전문가""","메리츠화재 최고리스크관리책임자(CRO), 메리츠금융지주 CRO 등을 역임했다. 메리...",https://www.paxnetnews.com/articles/96200
3,메리츠화재,3일 전,뉴스핌,"메리츠금융그룹, 정기 임원 인사 실시",[서울=뉴스핌] 김승동 기자 = 메리츠화재 본사 사옥 2020.09.03 0I087...,http://www.newspim.com/news/view/20221223000452
4,메리츠화재,3일 전,프라임경제,[인사] 메리츠화재,메리츠화재(000060)는 2023년 1월1일자로 임원 승진 인사를 단행했다고 23...,http://www.newsprime.co.kr/news/article.html?n...
...,...,...,...,...,...,...
83017,SK네트웍스,2022.07.01.,핀포인트뉴스,[SK그룹] 등락률 살펴보니…SK바이오팜 1.9% 상승 '미소',61%)▲8위 SK네트웍스 4200원(-0.71%)▲9위 SK 21만5000원(-1...,http://www.pinpointnews.co.kr/news/articleView...
83018,SK네트웍스,2022.08.24.,서울경제,"[기업공시 8월 23일]에브리봇, 1주당 1주 배정 무상증자 결정 등",▲SK네트웍스(001740)=전기차 충전 인프라 추진 사업 위해 728억 원 규모의...,https://www.sedaily.com/NewsView/269X7TCH4W
83019,SK네트웍스,2022.07.25.,Queen,"재계 세대교체 3·4세 등장 ""기존과 다른 스타일"" … 승계 작업 빨라질 듯",SK그룹에서는 최신원 전 SK네트웍스 회장의 장남인 최성환 사업총괄이 올해 SK네트...,http://www.queen.co.kr/news/articleView.html?i...
83020,SK네트웍스,2022.06.30.,ZDNet Korea,"SK, 35년 집념으로 ‘K-바이오’ 시대 이끈다","최태원 SK 회장(맨왼쪽)과 최재원 SK 수석부회장, 최창원 SK 디스커버리 부회장...",https://zdnet.co.kr/view/?no=20220630025900


In [16]:
df.to_excel('삼성증권_뉴스크롤링_'+str(start_index)+'_to_'+str(end_index)+'.xlsx')